### Search with Vectors
##### Idea: convert all tweets of each user into a vector and search for similarities
##### Note: the same user in different years is considered different

### Installation of libraries
##### It is advisable to do Kernel+restart if you have installed.
##### Note: it may take a while (only the first time)

In [ ]:
modules = ["pymongo", "pymongo[srv]","pprintpp","rich","tf_keras", "sentence-transformers", "ipywidgets"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

Data preparation

In [ ]:
datos = "usa162024.csv"
import pandas as pd
df = pd.read_csv(datos)
df["userYP"] = df["User"]+"_"+df.year.astype(str)+df["party"] # bertoldo becomes bertoldo_16, bertoldo_20, bertoldo_24
# Concatenate tweets per user
df_agg = df.groupby('userYP', as_index=False).agg({'Tweet': '.'.join})
df_agg

We prepare the transformer that will convert the texts into vectors

In [ ]:
from sentence_transformers import SentenceTransformer
#modelo = SentenceTransformer('all-MiniLM-L6-v2')
modelo = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
    

In [ ]:
from tqdm import tqdm 


cods = []
ids = []

for _,row in tqdm(df_agg.iterrows()):
    cod = modelo.encode(row["Tweet"]).tolist()
    cods.append(cod)
    ids.append(row["userYP"])

In [ ]:
df_encoding = pd.DataFrame({"User":ids,"embedding":cods})
path = "../data/"
df_encoding.to_csv(path+"encoding_user_768.csv",index=False)

In [ ]:
df_encoding